In [1]:
import cv2

In [2]:
import utils

In [3]:
import numpy

In [4]:
# yunet_blob_url = 'https://github.com/opencv/opencv_zoo/raw/refs/heads/main/models/face_detection_yunet/face_detection_yunet_2023mar_int8.onnx'
yunet_blob_url = 'https://github.com/opencv/opencv_zoo/raw/refs/heads/main/models/face_detection_yunet/face_detection_yunet_2023mar.onnx'

In [5]:
model_folder_path = '../model/'

In [6]:
# yunet_file_name='yunet_int8.onnx'
yunet_file_name='yunet.onnx'

In [7]:
utils.show_folders_and_files(model_folder_path)

../model/
  ├ dummy
  ├ yunet.onnx
  └ faces_and_rectangles_from_image


In [8]:
yunet_model_file_path = utils.fetch_file_stream(
    url=yunet_blob_url, 
    destination_folder_path=model_folder_path, 
    data_file_name=yunet_file_name,
    redownload=False,
)
utils.show_folders_and_files(model_folder_path)

../model/
  ├ dummy
  ├ yunet.onnx
  └ faces_and_rectangles_from_image


In [9]:
yunet_model_file_path

'../model/yunet.onnx'

In [10]:
def yunet_face_extractor_function_from_saving_model_file_path(model_file_path, image_shape=(640,480)): #.onnx ?
    face_extractor_model = cv2.FaceDetectorYN_create(model_file_path, "", (0, 0))
    face_extractor_model.setInputSize(image_shape)
    def faces_and_rectangles_from_image(image_array):
        # assert image_array.shape, f"image must have a shape -> numpy matrix : {image_array = } {type(image_array) = }"        
        _truc, faces = face_extractor_model.detect(image_array)
        # print(f"{faces = }")
        # print(f"output of the face extractor {_truc = } {faces = }")
        
        extracted_faces = []
        rectangles = []
        try:
            if faces is not None:
                for face in faces:
                    x, y, w, h = list( int(v) for v in face[:4] )
                    face_rectangle = image_array[y:y+h, x:x+w]
                    face_gray = cv2.cvtColor(face_rectangle, cv2.COLOR_BGR2GRAY)
                    face_resized = cv2.resize(face_gray, (48, 48))
                    extracted_faces.append(face_resized)
                    rectangles.append(top_left_and_bot_right:=((x, y), (x+w, y+h)))
        except:
            pass
                
        if faces is None:
            pass
        return extracted_faces, rectangles
    
    return faces_and_rectangles_from_image

In [11]:
def find_device_index():
    for device_index in range(0,200):
        video_capture = cv2.VideoCapture(device_index)
        if video_capture.isOpened():

            ok, frame = video_capture.read()
            if not ok:
                continue
            return device_index
        if not video_capture.isOpened():
            video_capture.release()
            continue
        # except:
        #     print(f"did not work {device_index = }")
        #     video_capture.release()
    raise Exception("could not find a working webcam device, try : `!cameractrls -L` if you are on systemd linux distro")

In [12]:
my_webcam_index = find_device_index()
my_webcam_index

[ WARN:0@0.433] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=48, req=-2140645888
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=48, req=-2140645888
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=48, req=-2140645888
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=48, req=-2140645888
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:138 queryUvcDeviceInfoList ioctl error return: 25
[ WARN:0@0.437] global obsensor_stream_channel_v4l2.cpp:82 xioctl ioctl: fd=48, req=-214064588

66

# display webcam output but with frame around faces

In [13]:
def frame_faces_from_video(video_file_path_or_index, model_file_path ,create_face_extractor_from_model_path_and_image_shape ):
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_file_path_or_index)
    webcam_shape = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)) , int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    faces_and_rectangles_from_image = yunet_face_extractor_function_from_saving_model_file_path(
        model_file_path=model_file_path, 
        image_shape=webcam_shape,
    )
    if not video_capture.isOpened():
        print(f"Cannot open camera {video_capture = }")
        return
    try:
        while True:
            ok, webcam_frame = video_capture.read()
            if not ok:
                print(f"end of video stream {ok = }")
                break
            faces, rectangles = faces_and_rectangles_from_image(webcam_frame)
            if rectangles:
                list(
                    cv2.rectangle(webcam_frame, top_left, bottom_right, color=(255, 0, 0), thickness=2) 
                    for top_left, bottom_right in rectangles
                )
            cv2.imshow('webcam_frame', webcam_frame) # showing webcam output
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as error:
        print(error)
        pass    
    video_capture.release()
    cv2.destroyAllWindows()

In [14]:
frame_faces_from_video(
    video_file_path_or_index=my_webcam_file_path, 
    model_file_path=yunet_model_file_path, 
    create_face_extractor_from_model_path_and_image_shape=yunet_face_extractor_function_from_saving_model_file_path,
)

NameError: name 'my_webcam_file_path' is not defined

# display only images of detected/extracted faces

In [15]:
def faces_from_video(video_file_path_or_index, model_file_path ,create_face_extractor_from_model_path_and_image_shape ):
    
    # Open the video file
    video_capture = cv2.VideoCapture(video_file_path_or_index)
    webcam_shape = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)) , int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    faces_and_rectangles_from_image = yunet_face_extractor_function_from_saving_model_file_path(
        model_file_path=model_file_path, 
        image_shape=webcam_shape,
    )
    if not video_capture.isOpened():
        print(f"Cannot open camera {video_capture = }")
        return
    try:
        while True:
            ok, webcam_frame = video_capture.read()
            if not ok:
                print(f"end of video stream {ok = }")
                break
            faces, rectangles = faces_and_rectangles_from_image(webcam_frame)
            if faces and rectangles:
                cv2.imshow('faces', numpy.hstack(faces))
            else:
                no_face_image = numpy.zeros((256,256))
                no_face_image = cv2.putText(
                    img=no_face_image,
                    text='no face', 
                    org=(10,40),# bottomLeftCornerOfText, 
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=1,
                    color=(255,255,255),
                    thickness=4,
                    bottomLeftOrigin=False,
                )
                cv2.imshow('faces', no_face_image)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    except Exception as error:
        print(error)
        pass    
    video_capture.release()
    cv2.destroyAllWindows()

In [16]:
faces_from_video(
    video_file_path_or_index=my_webcam_index, 
    model_file_path=yunet_model_file_path, 
    create_face_extractor_from_model_path_and_image_shape=yunet_face_extractor_function_from_saving_model_file_path,
)

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
